# Knuckledragger: A Low Barrier Proof Assistant
## Philip Zucker
### Draper Laboratory

<img src="https://raw.githubusercontent.com/philzook58/knuckledragger/main/docs/logo.webp" alt="drawing" width="200"/>

Try it out: <https://colab.research.google.com/github/philzook58/knuckeldragger/blob/master/examples/short_talk.ipynb>

In [ ]:
#! python3 -m pip install git+https://github.com/philzook58/knuckledragger.git
#! python3 -m kdrag.solvers install

# Today
- Z3 is Awesome
- Z3 is not perfect
- Enter Knuckledragger
- The Core System
- Feature Smorgasbord

# Motivation

- Z3 is Great
- Its python bindings are Great

In [18]:
import z3
x = z3.BitVec("x", 64)
z3.prove(z3.ForAll([x], x | x == x))

proved


In [19]:
x = z3.Real("x")
z3.prove(z3.ForAll([x], z3.Or(x < 0, x == 0, x > 0)))

proved


In [20]:
Nat = z3.Datatype("MyNat")
Nat.declare("Zero")
Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()

n,x,y = z3.Consts("n x y", Nat)

add = z3.Function("add", Nat, Nat, Nat)
add_def = z3.ForAll([x,y], add(x,y) == 
                    z3.If(Nat.is_Zero(x), 
                          y, 
                          Nat.Succ(add(Nat.pred(x), y))))

add_zero_left = z3.ForAll([n], add(Nat.Zero, n) == n)
z3.prove(z3.Implies(add_def, add_zero_left))

proved


## But it can't do everything.

- Quantifiers are dicey
- E-matching is incomplete
- No built in induction
- Shear scale

In [6]:
add_zero_right = z3.ForAll([n], add(n, Nat.Zero) == n)
z3.prove(z3.Implies(add_def, add_zero_right), timeout=3000)

failed to prove


Z3Exception: model is not available

# Enter Knuckledragger

- Interactive theorem prover as a library in Python
- Heavily and Shallowly based around Z3py
- Proof Objects vs Formulas

In [26]:
import kdrag as kd
import kdrag.smt as smt # smt is literally a reexporting of z3

x = smt.BitVec("x", 32)
or_idem = kd.prove(smt.ForAll([x], x | x == x))
or_idem
#type(or_idem)
#or_idem.thm


|- ForAll(x, x | x == x)

In [ ]:
# Knuckledragger support algebraic datatypes and induction
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()

In [ ]:
# We can define an addition function by cases
n,m = smt.Consts("n m", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], 
    kd.cond(
        (n.is_Zero, m),
        (n.is_Succ, Nat.Succ(add(n.pred, m)))
))

kd.notation.add.register(Nat, add)

In [9]:
add_zero_x = kd.prove(smt.ForAll([n], Nat.Zero + n == n), by=[add.defn])
add_succ_x = kd.prove(smt.ForAll([n,m], Nat.Succ(n) + m == Nat.Succ(n + m)), by=[add.defn])

In [ ]:
l = kd.Lemma(smt.ForAll([n], n + Nat.Zero == n))
#_n = l.fix()            
#l.induct(_n)              
#l.auto(by=[add.defn])
#l.auto(by=[add.defn])
#add_x_zero = l.qed()

# Inference Rules

- Hilbert style proof system
- LCF style kernel
    + Proof datatype
    + Small number of trusted constructors
- Proof objects are basically trees of recorded z3 calls

$$
\frac{
  t_1\ \text{True} \quad t_2\ \text{True} \quad \cdots \quad \lnot  (t_1 \land t_2 \land \cdots) \rightarrow t \ \text{unsat}
}{
  t\ \text{True}
}
\quad \text{prove}
$$

In [12]:
x = smt.Int("x")
xpos = kd.kernel.axiom(x > 0)
# ---------------------------
kd.kernel.prove(x > -1, by=[xpos])

|- x > -1

In [ ]:
kd.kernel.prove(x > -1, by=[xpos]).reason

[|- x > 0]

## But Not Quite the Whole Story
- Quantifier axiom schema
- Induction axiom schema
- User and theory specific schema

In [ ]:
vs, pf = kd.kernel.herb(smt.ForAll([x], x > 0))
pf

|- Implies(x!1195 > 0, ForAll(x, x > 0))

In [ ]:
pf = kd.prove(smt.ForAll([x], x > x - 1))
kd.kernel.instan([smt.IntVal(42)], pf)

|- 42 > 42 - 1

In [ ]:
import kdrag.theories.nat as nat
n = smt.Const("n", nat.Nat)
P = smt.Function("P", nat.Nat, smt.BoolSort())
kd.kernel.induct_inductive(n, P)

|- Implies(And(P(Z),
            ForAll(pred!1202,
                   Implies(P(pred!1202), P(S(pred!1202))))),
        P(n))

# Backwards Tactics

- Mutable `Lemma` object with list of goals
- tactic methods
    + Pop goal
    + record `Proof` lemmas
    + push new goals
- `l.qed()` assembles lemmas via `prove` call

# Smorgasbord


## Theories


In [ ]:
import kdrag.theories.real as real
import kdrag.theories.bool as bool_
import kdrag.theories.bitvec as bitvec
import kdrag.theories.seq as seq
import kdrag.theories.nat as nat
import kdrag.theories.int as int_
import kdrag.theories.list as list_
import kdrag.theories.option as option
import kdrag.theories.set as set_
import kdrag.theories.real.complex as complex
import kdrag.theories.algebra.group as group
import kdrag.theories.algebra.lattice
import kdrag.theories.algebra.ordering
import kdrag.theories.fixed
import kdrag.theories.float
import kdrag.theories.real.arb
import kdrag.theories.real.sympy
import kdrag.theories.real.vec

# shallow embeddings of logics
import kdrag.theories.logic.intuitionistic
import kdrag.theories.logic.temporal

Admitting lemma ForAll([t, s], mul(expi(t), expi(s)) == expi(t + s))


       norm2(u) ==
       0 + x0(u)*x0(u) + x1(u)*x1(u) + x2(u)*x2(u)) to ForAll(u,
       norm2(u) == x0(u)*x0(u) + x1(u)*x1(u) + x2(u)*x2(u))


### Quickcheck

- Uses python hypothesis to quickcheck goals
- Relate implementations to axiomatization

In [ ]:
import kdrag.hypothesis as hyp
import kdrag.theories.nat as nat
n = smt.Const("n", nat.Nat)
hyp.quickcheck(smt.ForAll([n], n + nat.Z == n))

In [ ]:
hyp.quickcheck(smt.ForAll([n], n + nat.Z == n + nat.one))

AssertionError: ('Found a counterexample', [])

### Search

- Finding the right lemmas is half the battle

In [ ]:
x = smt.Real("x")
kd.search(real.cos(x)**2)

{('kdrag.theories.real.pythag_1',
  |- ForAll(x, cos(x)**2 == 1 - sin(x)**2)): [x],
 ('kdrag.theories.real.cos_neg', |- ForAll(x, cos(-x) == cos(x))): [x]}

### Rewriting

- Controlled rewriting of Z3 expressions

In [ ]:
x,y,z = smt.Reals("x y z")
unit = kd.prove(smt.ForAll([x], x + 0 == x))
mul_zero = kd.prove(smt.ForAll([x], x * 0 == 0))
kdrag.rewrite.rewrite(0 + x + y*0 + z + 0 + x*0, [unit, mul_zero])

0 + x + z

In [ ]:
smt.simplify(0 + x + y*0 + z + 0 + x*0)

x + z

### Syntactic Utilities

- Locally nameless combinators
- Unification and pattern matching
- Term orderings
- Pattern matching

In [35]:
x,y,z = smt.Ints("x y z")
kd.utils.open_binder(smt.ForAll([x,y], x + y == y + x))

([X!110, Y!111], X!110 + Y!111 == Y!111 + X!110)

In [36]:
kd.utils.unify([x,y,z], x + (x + z), y + (x + 0))

{z: 0, x: y}

In [46]:
n = smt.Const("n", Nat)
n.match_(
        (Nat.Zero, m),
        (Nat.Succ(n), Nat.Succ(add(n, m)))
)

If(is(Zero, n),
   m,
   If(is(Succ, n), Succ(add(pred(n), m)), unreachable!124))

### Knuth Bendix Completion

In [ ]:
import kdrag.solvers.kb as kb
# https://www.philipzucker.com/knuth_bendix_knuck/
T = smt.DeclareSort("AbstractGroup")
x,y,z = smt.Consts("x y z", T)
e = smt.Const("a_e", T)
inv = smt.Function("c_inv", T, T)
mul = smt.Function("b_mul", T, T, T)
kd.notation.mul.register(T, mul)
kd.notation.invert.register(T, inv)
E = [
    smt.ForAll([x], e * x == x),
    smt.ForAll([x], inv(x) * x == e),
    smt.ForAll([x,y,z], (x * y) * z == x * (y * z)),
]
kb.huet(E, order=kd.rewrite.lpo)

[RewriteRule(vs=[Z!10564, X!10565], lhs=c_inv(b_mul(X!10565, Z!10564)), rhs=b_mul(c_inv(Z!10564), c_inv(X!10565)), pf=None),
 RewriteRule(vs=[Z!7894, X!7895], lhs=b_mul(X!7895, b_mul(c_inv(X!7895), Z!7894)), rhs=Z!7894, pf=None),
 RewriteRule(vs=[X!7885], lhs=c_inv(c_inv(X!7885)), rhs=X!7885, pf=None),
 RewriteRule(vs=[], lhs=c_inv(a_e), rhs=a_e, pf=None),
 RewriteRule(vs=[X!7880], lhs=b_mul(X!7880, c_inv(X!7880)), rhs=a_e, pf=None),
 RewriteRule(vs=[X!7611], lhs=b_mul(X!7611, a_e), rhs=X!7611, pf=None),
 RewriteRule(vs=[Z!7529, X!7530], lhs=b_mul(c_inv(X!7530), b_mul(X!7530, Z!7529)), rhs=Z!7529, pf=None),
 RewriteRule(vs=[X!7493], lhs=b_mul(a_e, X!7493), rhs=X!7493, pf=None),
 RewriteRule(vs=[X!7490], lhs=b_mul(c_inv(X!7490), X!7490), rhs=a_e, pf=None),
 RewriteRule(vs=[X!7485, Y!7486, Z!7487], lhs=b_mul(b_mul(X!7485, Y!7486), Z!7487), rhs=b_mul(X!7485, b_mul(Y!7486, Z!7487)), pf=None)]

### External Solvers
- Print to smt and tptp
- Eprover, Vampire

In [ ]:
s = kd.solvers.EProverTHFSolver()
for eq in E:
    s.add(eq)
s.add(smt.ForAll([x], x * e != x))
s.check()
print(s.res.stdout.decode())

# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)
# Starting new_ho_10 with 1500s (5) cores
# Starting ho_unfolding_6 with 300s (1) cores
# Starting sh4l with 300s (1) cores
# Starting ehoh_best_nonlift_rwall with 300s (1) cores
# ho_unfolding_6 with pid 1176906 completed with status 0
# Result found by ho_unfolding_6
# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)
# Starting new_ho_10 with 1500s (5) cores
# Starting ho_unfolding_6 with 300s (1) cores
# No SInE strategy applied
# Search class: HUUPS-FFSF21-SFFFFFNN
# Scheduled 6 strats onto 1 cores with 300 seconds (300 total)
# Starting new_ho_10 with 163s (1) cores
# new_ho_10 with pid 1176910 completed with status 0
# Result found by new_ho_10
# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)
# Starting new_ho_10 with 1500s (5) cores
# Starting ho_unfolding_6 with 300s (1

In [ ]:
s = kd.solvers.VampireSolver()
for eq in E:
    s.add(eq)
s.add(smt.ForAll([x], x * e != x))
s.check()
print(s.res.stdout.decode())

(set-logic ALL)

(declare-sort AbstractGroup 0)

;;declarations

(declare-fun b_mul_2d4 (AbstractGroup AbstractGroup) AbstractGroup)

(declare-fun a_e_22a () AbstractGroup)

(declare-fun c_inv_34e (AbstractGroup) AbstractGroup)

;;axioms

(assert (forall ((X!10954_398 AbstractGroup)) (= (b_mul_2d4 a_e_22a X!10954_398) X!10954_398)))

(assert (forall ((X!10955_33c AbstractGroup)) (= (b_mul_2d4 (c_inv_34e X!10955_33c) X!10955_33c) a_e_22a)))

(assert (forall ((X!10956_398 AbstractGroup) (Y!10957_33c AbstractGroup) (Z!10958_322 AbstractGroup)) (= (b_mul_2d4 (b_mul_2d4 X!10956_398 Y!10957_33c) Z!10958_322) (b_mul_2d4 X!10956_398 (b_mul_2d4 Y!10957_33c Z!10958_322)))))

(assert (forall ((X!10959_3a1 AbstractGroup)) (distinct (b_mul_2d4 X!10959_3a1 a_e_22a) X!10959_3a1)))

(check-sat)

b'WARNING Broken Constraint: if inline_let(on) has been set then newcnf(off) is equal to on\nWARNING Broken Constraint: if inline_let(on) has been set then newcnf(off) is equal to on\nWARNING Broken Constraint

### Reflection
- An applicative subset of Python into z3
- Evaluation from z3 into python

In [44]:
import kdrag.reflect as reflect

@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

In [43]:
reflect.eval_(Nat.Zero + Nat.Zero)

Zero()

### Other Subsystems
- Operator Overloading
- Typeclasses and Generics
- Sympy interop
- Arb
- Inductive Relations
- Prolog

# Future Work
- Textual proof certificates and standalone checkers
- Fill out more theories
- Software Foundations
- Extraction to C, Rust, Lean, Verilog
- E-graph rewriting
- Applications
    + Binary Verification via Ghidra
    + Verilog importing via yosys

# Conclusion
- A low barrier interactive theorem prover based around z3
- http://www.kdrag.com
- Thanks for listening!
